<a href="https://colab.research.google.com/github/shiivya/P3-Code/blob/main/Final_P3_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#dispenses random container
def dispense_container():
  #generates random number to dispense random container type
  container_num = random.randint(1,6)
  #returns listi with container information [material, mass, binID]
  container_properties = table.dispense_container(container_num, True) 
  return container_properties 


#loads bottle onto the hopper
def load_container(num_hopper, last_binID):

    #moves q-arm to table with dispensed bottle
    arm.move_arm(0.64,0,0.23)
    time.sleep(2)

    #q-arm grips bottle
    arm.control_gripper(40)
    time.sleep(2)
    
    #places bottle on specific position on hopper
    if (num_hopper==1):
        if (last_binID[4]=="1"): #specific hopper position for metal can
            arm.move_arm(0.012, -0.617, 0.531)
        else:
            arm.move_arm(0.012, -0.617, 0.501)
            time.sleep(2)
            arm.rotate_wrist(15)
    elif (num_hopper==2):
        arm.move_arm(0.012, -0.505, 0.501)
    else:
        arm.move_arm(0.012, -0.45, 0.461)
                
    time.sleep(2)

    #q-arm released bottle
    arm.control_gripper(-40)
    time.sleep(2)

    #prevents q-arm from knocking over bottles placed on hopper
    arm.move_arm(0.0, -0.3, 0.501)
    time.sleep(2)
    arm.home()
 
#transfer container function
def transfer_container(target_bin):
    #activate sensors
    bot.activate_line_following_sensor()
    bot.activate_color_sensor()
    bot.activate_ir_sensor()

# Determine the target color based on the target bin.
if target_bin == "Bin01":
    target_color = [1, 0, 0]  # red
elif target_bin == "Bin02":
    target_color = [0, 0, 1]  # blue
elif target_bin == "Bin03":
    target_color = [0, 1, 0]  # green
elif target_bin == "Bin04":
    target_color = [0, 0, 0]  # black

# Move the robot to the target bin and deposit the container.
if target_bin == "Bin01":
    # Move the robot to the target position.
    while round(bot.position()[0], 2) != 1.10 and round(bot.position()[1], 2) != 0.69:
        if bot.line_following_sensors() == [1, 1]:
            bot.forward_distance(0.05)
        else:
            bot.rotate(-2)

    # Check if the container matches the target color and deposit if it does.
    color = bot.read_color_sensor()[0]
    if color == target_color:
        print("Arrived at Bin 01")
        bot.rotate(-2)
        deposit_container()

elif target_bin == "Bin02":
    # Move the robot to the target position.
    tolerance = 0.1  # Set the tolerance.
    target_position = [0.62, 0.72]  # Set the target position.
    while True:
        current_position = bot.position()
        if abs(current_position[0] - target_position[0]) < tolerance and abs(current_position[1] - target_position[1]) < tolerance:
            break
        if bot.line_following_sensors() == [1, 1]:
            bot.forward_distance(0.05)
        else:
            bot.rotate(-2)

    # Check if the container matches the target color and deposit if it does.
    color = bot.read_color_sensor()[0]
    if color == target_color:
        print("Arrived at Bin 02")
        deposit_container()

elif target_bin == "Bin03":
    # Move the robot to the target position.
    tolerance = 0.1  # Set the tolerance.
    target_position = [0.56, -0.72]  # Set the target position.
    while True:
        current_position = bot.position()
        if abs(current_position[0] - target_position[0]) < tolerance and abs(current_position[1] - target_position[1]) < tolerance:
            break
        if bot.line_following_sensors() == [1, 1]:
            bot.forward_distance(0.05)
        else:
            bot.rotate(-2)

    # Check if the container matches the target color and deposit if it does.
    color = bot.read_color_sensor()[0]
    if color == target_color:
        print("Arrived at Bin 03")
        deposit_container()

elif target_bin == "Bin04":
    # Move the robot to the target position.
    tolerance = 0.1  # Set the tolerance.
    target_position = [1.10, -0.69]  # Set the target position.
    while True:
        current_position = bot.position()
        if abs(current_position[0] - target_position[0]) < tolerance and abs(current_position[1] - target_position[1]) < tolerance:
            break
        if bot.line_following_sensors() == [1,1]:
            bot.forward_distance(0.05)
        else:
            bot.rotate(-2)

    # Check if the container matches the target color and deposit if it does.
    color = bot.read_color_sensor()[0]
    if color == target_color:
        print("arrived! bin 4") 
        deposit_container()

#deactivate sensors
bot.deactivate_color_sensor()
bot.deactivate_ir_sensor()

    
def deposit_container():
  time.sleep(2)
  bot.activate_linear_actuator()
  time.sleep(2)
  #tilts hopper to deposit bottles into correct bin
  bot.dump()
  return_home()

def return_home():
    bot.activate_line_following_sensor()

    # Loop until the robot reaches its home position.
    while True:
        # Get the current x and y coordinates of the robot's position.
        x = round(bot.position()[0], 2)
        y = round(bot.position()[1], 2)

        # Check if the robot has reached its home position.
        if 1.40 <= x <= 1.50 and 0 <= y <= 0.10:
            break

        # If the line following sensors detect a line, move the robot forward.
        # Otherwise, rotate the robot to the left.
        if bot.line_following_sensors() == [1, 1]:
            bot.forward_distance(0.05)
        else:
            bot.rotate(-2)

    # Deactivate the line following sensor and stop the robot.
    bot.deactivate_line_following_sensor()
    bot.stop()


def main():
    new_container= dispense_container()
    container_mass= new_container[1]
    target_bin= new_container[2]
    while True:
       #resets hopper mass and num containers on hopper everytime bottles has been transferred
        hopper_mass=0
        num_hopper=1
        load_container(num_hopper, target_bin)
        hopper_mass+=container_mass
        while True:
            next_container= dispense_container()
            next_mass= next_container[1]
            next_bin= next_container[2]
            print(next_bin + " == " + target_bin)
            if (num_hopper<3 and hopper_mass<90 and target_bin== next_bin):
                hopper_mass+= next_mass
                num_hopper+=1
                load_container(num_hopper, target_bin)

            #does not load container if one of the conditions is not satisfied
            else:
                break
        print("Going to " + target_bin)

        #transfers bottles to correct bin
        transfer_container(target_bin)
        target_bin= next_bin
        return_home()
        time.sleep(2)
